In [14]:
import pandas as pd

In [15]:
sampleTripMilestones = pd.DataFrame.from_csv('./data/2014_SF_Top_TripMileStones.csv')

x = sampleTripMilestones.reset_index()
routeDict={}
for index, row in x.iterrows():
    routeInfo={}
    routeInfo['totalDistance'] = row['totalDistance']
    routeInfo['milestoneCount'] = row['milestoneCount']
    routeInfo['mileStoneLats'] = row['mileStoneLats']
    routeInfo['mileStoneLngs'] = row['mileStoneLngs']
    
    routeDict[(row['startStationId'], row['endStationId'])] = routeInfo

len(routeDict)

29

In [16]:
import datetime
import csv
import ast

def roundTimeToGivenSecondLapse(dt=None, roundTo=60):
   """Round a datetime object to any time laps in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = datetime.datetime.now()
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)



def getTripMileStoneTimes(index,row):
    tripMileStaoneTimesRow={}
    mileStoneCount = routeDict[(row['startStationId'], row['endStationId'])]['milestoneCount']
    mileStoneTimes= []
#     print  row['Start Date'], mileStoneCount, row['Duration']

    tripMileStaoneTimesRow['tripID'] = index
    tripMileStaoneTimesRow['startTime'] = row['Start Date']
    tripMileStaoneTimesRow['duration'] = row['Duration']
    tripMileStaoneTimesRow['startStationId'] = row['startStationId']
    tripMileStaoneTimesRow['endStationId'] = row['endStationId']
    
    curMileStoneIndex = 0
    while (curMileStoneIndex < mileStoneCount):
#         print row['Start Date'], (row['Duration'] * curMileStoneIndex)/mileStoneCount 
#         print row['Start Date'], (row['Duration'] * curMileStoneIndex)/mileStoneCount 
        currentMilestoneDuration= (row['Duration'] * curMileStoneIndex)/mileStoneCount 
        mileStoneTime = datetime.datetime.strptime(row['Start Date'], "%m/%d/%y %H:%M") + datetime.timedelta(seconds=currentMilestoneDuration)
#         print roundTimeToGivenSecondLapse(mileStoneTime,10)

        currentMilestoneTime = roundTimeToGivenSecondLapse(mileStoneTime,10)
        currentMilestoneTimeStr = currentMilestoneTime.strftime("%m/%d/%y %H:%M:%S")
        mileStoneTimes.append(currentMilestoneTimeStr)
        curMileStoneIndex = curMileStoneIndex + 1
#     print(mileStoneTimes)
    tripMileStaoneTimesRow['mileStoneTimes'] = mileStoneTimes
    return tripMileStaoneTimesRow



In [17]:

tripsOfTopRoutes = pd.DataFrame.from_csv('./data/2014_SF_TripsOfTop30Routes_Aug2013.csv')
len(tripsOfTopRoutes)


167

In [18]:
toCSV=[]
for index, row in tripsOfTopRoutes.iterrows():
    if (row['startStationId'] != row['endStationId']):
        tripMilestoneTimesRow = getTripMileStoneTimes(index,row)
        toCSV.append(tripMilestoneTimesRow)

# keys = milestoneRow.keys()
keys=[
 'tripID',
 'startTime',
 'duration',
 'startStationId',
 'endStationId',
 'mileStoneTimes']
with open('./data/2014_SF_TripsOfTop30Routes_Aug2013_Milestones.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)
print 'done'

done
